In [ ]:
import torch
import torch.nn as nn
import os
from mmdet.apis import init_detector, inference_detector, show_result
from mmdet.models import build_detector
import mmcv
from mmcv.runner import load_checkpoint
from mmdet.datasets.pipelines import Compose
from mmcv.parallel import collate, scatter
import numpy as np
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3"  # specify which GPU(s) to be used
device='cuda:0'

In [2]:
print(torch.cuda.device_count(),torch.cuda.current_device())

3 0


In [3]:
import run

In [4]:
feature_extractor = run.get_feature_extractor(config_file='configs/tbtc_fater_rcnn_voc.py',
                          checkpoint_file='checkpoints/faster_rcnn_x101_64x4d_fpn_1x20200324-ba5926a5.pth',
                          device='cuda:0')

In [3]:
idx = 1 # 0 for faster rcnn, 1 for retinanet 
config_file = ['configs/tbtc_fater_rcnn_voc.py','tbtc_retinanet_voc.py'][idx]
checkpoint_file = ['checkpoints/faster_rcnn_x101_64x4d_fpn_1x20200324-ba5926a5.pth',
                   'retinanet_x101_64x4d_fpn_1x20200322-53c08bb4.pth'][idx]
# config = mmcv.Config.fromfile(config_file)
# model = build_detector(config.model)
# model = model.to(device)
model = init_detector(config_file, checkpoint_file,device=device)
# build the model from a config file and a checkpoint file


In [21]:
list(model.children())

[ResNeXt(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): Bottleneck(
       (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
       (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace)
       (downsample): Sequential(
         (0): Conv2d(64, 256, kernel_

In [4]:
class LoadImage(object):

    def __call__(self, results):
        if isinstance(results['img'], str):
            results['filename'] = results['img']
        else:
            results['filename'] = None
        img = mmcv.imread(results['img'])
        results['img'] = img
        results['img_shape'] = img.shape
        results['ori_shape'] = img.shape
        return results




def get_img(cfg,img):
    # build the data pipeline
    test_pipeline = [LoadImage()] + cfg.data.test.pipeline[1:]
    test_pipeline = Compose(test_pipeline)
    # H x W x C -> C x H x W and expand an dim
    data = dict(img=img)
    data = test_pipeline(data)
    data = scatter(collate([data], samples_per_gpu=1), [device])[0]
    return data
img = 'demo/000454_5.jpg'
config = mmcv.Config.fromfile(config_file)
data = get_img(config,img)

In [5]:
data

{'img_metas': [[{'filename': 'demo/000454_5.jpg',
    'ori_shape': (800, 800, 3),
    'img_shape': (800, 800, 3),
    'pad_shape': (800, 800, 3),
    'scale_factor': 1.0,
    'flip': False,
    'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32),
     'std': array([58.395, 57.12 , 57.375], dtype=float32),
     'to_rgb': True}}]],
 'img': [tensor([[[[-0.9534, -0.9877, -0.9534,  ...,  0.3823,  0.3652,  0.3823],
            [-0.8335, -0.8164, -0.7822,  ...,  0.3309,  0.3309,  0.3652],
            [-0.7137, -0.6452, -0.6281,  ...,  0.2796,  0.2967,  0.3481],
            ...,
            [-0.1828, -0.0972, -0.1143,  ...,  0.3481,  0.3481,  0.2453],
            [-0.1828, -0.1486, -0.0458,  ...,  0.3138,  0.3138,  0.1254],
            [ 0.0056,  0.0569,  0.2967,  ...,  0.4851,  0.3652,  0.2453]],
  
           [[-0.8277, -0.8627, -0.8277,  ...,  0.4853,  0.4853,  0.5028],
            [-0.7052, -0.6877, -0.6527,  ...,  0.4328,  0.4503,  0.4853],
            [-0.5826, -0.

In [24]:
with torch.no_grad():
    feat = model.backbone(data['img'][0])
    fpn_feat = model.neck(feat) 
    rpn_outs = model.rpn_head(fpn_feat) # RPN -> (loss_rpn_cls, loss_rpn_bbox)
    roi_outs = model.bbox_roi_extractor(rpn_outs)
    

TypeError: forward() missing 1 required positional argument: 'rois'

In [22]:
with torch.no_grad():
    model.ResNeXt()

SyntaxError: unexpected EOF while parsing (<ipython-input-22-6f21c0975997>, line 1)

Sequential(
  (0): ResNeXt(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
     

AttributeError: 'Sequential' object has no attribute 'cfg'

In [ ]:
def get_model(config, max_num=1000):
    '''build the object detection model and load checkpoint
    Args:
        1. config      : the cfg from init_config().
        2. max_num  : the maximum proposals of RPN, default = 1000 
                      ,which means the 1000 proposals are reseverd.
    Return: model and cfg
    '''
    # build model
    config.test_cfg['rpn']['max_num'] = max_num
    od_model = build_detector(config.model, train_cfg=None, test_cfg=config.test_cfg)
    # load checkpoint
    checkpoint = load_checkpoint(od_model, checkpoint_path,  map_location='cpu')
    return od_model, config

# todo 检查data格式，是否是列表
def get_img(cfg,img_meta):
    # build the data pipeline
    test_pipeline = [LoadImage()] + cfg.data.test.pipeline[1:]
    test_pipeline = Compose(test_pipeline)
    # H x W x C -> C x H x W and expand an dim
    img = torch.from_numpy(np.transpose(img, (2, 0, 1))).expand(1,-1,-1,-1)
    data = dict(img=img)
    data = scatter(collate([data], samples_per_gpu=1), [device])[0]
    return data


def get_roi_feat(config, img_meta, img, model):
    '''Get a 7x7x256 ROI feature from object detection model
    Args:
        1. config     : the cfg.
        2. img_meta   : the image meta data from pre-prepared img_meta database.
        3. img        : the image tensor.
        4. model      : the object detection model.
        
    Return: 
    '''
    with torch.no_grad(): 
        # Input should be Tensor, output a tuple with 4 tensor.
        # feat[0-3].shape:
        # 1 256 336 192
        # 1 512 168 96
        # 1 1024 84 48
        # 1 2048 42 24
        feat = model.backbone(img) # ResNet -> tuple

        # Input is a tuple，output a tuple with 5 tensor
        # fpn_feat[0-4].shape:
        # 1 256 336 192
        # 1 256 168 96
        # 1 256 84 48
        # 1 256 42 24
        # 1 256 21 12 # the feature is usefulless
        fpn_feat = model.neck(feat) # FPN -> tuple

        # Input is a tuple, output is a tuple(loss_rpn_cls,loss_rpn_bbox)
        # loss_rpn_cls, loss_rpn_bbox
        # 1 3/12 336 192
        # 1 3/12 168 96
        # 1 3/12 84 48
        # 1 3/12 42 24
        # 1 3/12 21 12
        rpn_outs = model.rpn_head(fpn_feat) # RPN -> (loss_rpn_cls, loss_rpn_bbox)

        # get 'max_num' proposals for each feature maps with different scales. proposal_list[0].shape -> torch.Size(['max_num', 5])
        # eg. >>proposal_list[0][0]
        # tensor([559.7507,  90.6900, 579.0510, 120.9980, 0.6428]) which is (xmin, ymin, xmax, ymax) and score
        proposal_cfg =config.test_cfg.get('rpn_proposal', config.test_cfg.rpn)
        proposal_cfg['max_num'] = 16 # we only want the proposals with the top 16 score
        proposal_inputs = rpn_outs + (img_meta, proposal_cfg)
        proposal_list = model.rpn_head.get_bboxes(*proposal_inputs)    

        # convert proposals to rois
        proposals = proposal_list[0].cuda()
        rois = bbox2roi([proposals])

        # the ROI features after ROIAlign. roi_feats.size() -> 'max_num'x256x7x7
        fpn_feat = [fpn_feat[0].cuda(),fpn_feat[1].cuda(),fpn_feat[2].cuda(),fpn_feat[3].cuda(),fpn_feat[4].cuda()]
        roi_feats = model.bbox_roi_extractor(
            fpn_feat[:len(model.bbox_roi_extractor.featmap_strides)],rois)

        # random select a roi_feats
        rand_idx = random.sample(range(proposal_cfg['max_num']), 1)
        roi_feat = roi_feats[rand_idx,:,:,:].expand(1,-1,-1,-1) # 1x256x7x7

    return roi_feat